In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
#Code to display the entire rows and colomns for ease of analysis

pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)

In [4]:
df = pd.read_csv('dataset/toronto_neighborhood.csv')
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
4,M9A,Queen's Park,Not assigned,43.667856,-79.532242
